# Tutorial parte 1

## Visualização dos dados usando métodos do DataFrame

In [ ]:
%matplotlib widget

from exercicio_holoviz import get_data_parquet

df = get_data_parquet()
df.loc[df['nom_usina'] == 'Itaipu 60 Hz'].plot.line(x='din_instante', y='val_geracao')



## Mesma visualização usando o hvplot

In [ ]:
import hvplot.pandas
from exercicio_holoviz import get_data_parquet

df = get_data_parquet()
df.loc[df['nom_usina'] == 'Itaipu 60 Hz'].hvplot.line(
    x="din_instante",
    y="val_geracao",
    value_label="MW",
    legend="top",
    height=500,
    width=1280,
)

## Visualizando todos os dados

In [ ]:
import hvplot.pandas
from exercicio_holoviz import get_data_parquet

df = get_data_parquet()

df.loc[df["nom_usina"] == "Itaipu 60 Hz"].hvplot.line(
    x="din_instante",
    y="val_geracao",
    value_label="MW",
    legend="top",
    height=500,
    width=1280,
    rasterize=True,
    cnorm="eq_hist",
)

## Visualizando os dados com Explorer

In [ ]:
import hvplot
import panel as pn
from exercicio_holoviz import get_data_parquet

pn.extension()

hvexplorer = hvplot.explorer(get_data_parquet())
hvexplorer

In [ ]:
import hvplot.pandas
from exercicio_holoviz import get_data_parquet

df = get_data_parquet()
df.hvplot(
    groupby=['nom_usina'],
    kind='line',
    x='din_instante',
    y=['val_geracao'],
)

# Tutorial parte 2

## Mostrando uma imagem bem simples de um agrupamento

In [ ]:
from exercicio_holoviz import get_data_parquet

grp_hlz = get_data_parquet().loc[:, ["din_instante", "val_geracao"]].groupby("din_instante").sum().hvplot(title="Geração diária do Brasil")
grp_hlz

In [ ]:
from exercicio_holoviz import get_data_parquet

grp_hlz_mes = get_data_parquet().loc[:, ["din_instante", "val_geracao"]].set_index("din_instante").resample("M").sum().hvplot(title="Geração mensal do Brasil")
grp_hlz_mes


In [ ]:
from exercicio_holoviz import get_data_parquet

grp_hlz = get_data_parquet().loc[:, ["din_instante", "val_geracao"]].groupby("din_instante").sum().hvplot(title="Geração diária do Brasil")
grp_hlz_mes = get_data_parquet().loc[:, ["din_instante", "val_geracao"]].set_index("din_instante").resample("M").sum().hvplot(title="Geração mensal do Brasil")

plot = (grp_hlz + grp_hlz_mes).cols(1)

# Zoom independente de cada gráfico
plot.opts(shared_axes=False)
plot


# Tutorial parte 3

In [2]:
import hvplot.pandas
from exercicio_holoviz import get_data_parquet

df = get_data_parquet()
graph_usina = df.loc[df["nom_usina"] == "Itaipu 60 Hz"].set_index("din_instante").hvplot(
    y="val_geracao", kind="bar", responsive=True, min_height=500, title="Geracao Itaipu"
)

graph_regiao = df.query('nom_subsistema == "SUL"').loc[:, ["din_instante", "val_geracao"]].groupby(
    "din_instante"
).sum().hvplot(
    y="val_geracao", kind="bar", responsive=True, min_height=500, title="Geracao Sul"
)

link = graph_usina + graph_regiao
link

:Layout
   .Bars.I  :Bars   [din_instante]   (val_geracao)
   .Bars.II :Bars   [din_instante]   (val_geracao)

In [12]:
import hvplot.pandas
from exercicio_holoviz import get_data_parquet
import holoviews as hv
import panel as pn

df = get_data_parquet().loc[:, ["din_instante", "nom_usina", "val_geracao"]]

pn.extension()

lista_opcoes_usinas = list(df["nom_usina"].unique())
lista_opcoes_usinas.sort()
select = pn.widgets.Select(name="Usina", options=lista_opcoes_usinas)

df_interativo = df.interactive()

df_interativo[df_interativo["nom_usina"] == select].hvplot(
    x="din_instante", y="val_geracao", title=f"Geracao Usina"
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'224f4c89-455a-46d3-9647-999cc0280042': {'version…

# Tutorial parte 4

In [16]:
import panel as pn
import holoviews as hv
import colorcet as cc
import hvplot.pandas  # noqa
from exercicio_holoviz import get_data_parquet


def get_grafico():
    df = get_dataframe()
    df_interativo = df.interactive()
    filtro_data = pn.widgets.DatePicker(name='Data', value=df.index.min().date())

    df_interativo_filtrado = df_interativo[filtro_data:]

    return df_interativo_filtrado.hvplot(responsive=True, min_height=400)


def get_dataframe():
    df = (
        get_data_parquet()
        .query("nom_usina.str.contains('Itaipu 50')")
        .loc[:, ["din_instante", "val_geracao"]]
        .set_index("din_instante")
    )

    return df


pn.extension("tabulator", template="material", sizing_mode="stretch_width")

titulo = pn.pane.Markdown("# Teste dados geração usina")
grafico = get_grafico()
dashboard = pn.Column(titulo, grafico)

dashboard.servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'e7ed3048-72e1-4142-96e7-9f6512debda7': {'version…